# Doc2vecの文書ベクトルを使用し、scikit-learnでコサイン類似検索




- Wikipediaの文書からDoc2Vecモデルを生成


- scikit-learnのcosine-simularityを使用して類似検索


nosetestsの質問文を使用してテストしたところ、４問中３問が正解、という結果を得ました。

## (1) Wikipediaコンテンツファイルから全文書を抽出

レポート <a href="31-Wikipedia-contents-csv.ipynb"><b>31-Wikipedia-contents-csv.ipynb</b></a> の手順にて、いったんローカルPCにCSVファイル化しておきます。

## (2) Wikipedia文書を学習

レポート <a href="27-Create-doc2vec-model-wiki.ipynb"><b>27-Create-doc2vec-model-wiki.ipynb</b></a> の手順にて生成したDoc2Vecモデルファイルをロードして使用します。

上記手順では、Wikipedia文書のみを使用し、ボキャブラリ／単語ベクトルの生成および学習を行い、モデルをファイル保存しています。

In [1]:
'''
    環境準備
'''
import sys
import os

import numpy as np
import pandas as pd
 
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)
if learning_dir not in sys.path:
    sys.path.append(learning_dir)

In [2]:
from gensim import models
from gensim.models.doc2vec import Doc2Vec

def doc2vec_model_path(dm):
    model_path = 'prototype/better_algorithm/doc2vec.wikipedia.PV%d.model' % dm
    return model_path

In [3]:
'''
    あらかじめ学習したモデルのファイルをロード
    dm = 0 : DBoWを使用したモデル
'''
dm = 0
loaded_model_dbow = models.Doc2Vec.load(doc2vec_model_path(dm))

print('Document vector size=%d' % (len(loaded_model_dbow.docvecs)))

Document vector size=431680


## (3) my-opeの文書を、Wikiから生成したモデルにより文書ベクトル化する関数

Wikipedia文書だけで学習されたDoc2Vecモデルを使用し、my-ope文書（質問文）をベクトル化します。

In [4]:
import numpy as np

from learning.core.learn.learning_parameter import LearningParameter
from learning.core.datasource import Datasource

_bot_id = 13 # toyotsu_human.csv
attr = {
    'include_failed_data': False,
    'include_tag_vector': False,
    'classify_threshold': 0.5,
    'algorithm': LearningParameter.ALGORITHM_LOGISTIC_REGRESSION,
    'params_for_algorithm': {'C': 140},
    'excluded_labels_for_fitting': None
}
learning_parameter = LearningParameter(attr)

In [5]:
from learning.core.nlang import Nlang

In [6]:
def get_document_vector(question, model, warning):
    '''
        question: 
            分かち書きされていない文書
        model:
            Doc2Vecの学習済みモデル
            （検証時は品詞を落としていないWikipedia文書からモデルを生成）

        inferred_vector:
            文書を分かち書きしたコーパスから、
            Doc2Vecの学習済みモデルを使用して
            生成される類似文書ベクトル
            （learning.core.nlang.Nlangの仕様に従い、
            　一部品詞が落とされます。）

            非常にサンプル数が多いので、類似文書ベクトル生成時の
            学習レート[alpha]を小さく設定し、かつ、
            反復回数[steps]を大幅に増加させています。

        warning:
            Trueを指定時、コーパスに含まれる単語が
            モデル内のWord2Vecボキャブラリにない場合、
            警告を表示する
    '''
    corpus = Nlang.split(question).split()
    inferred_vector = model.infer_vector(corpus, alpha=0.01, min_alpha=0.0001, steps=1000)
    
    if warning:
        for c in corpus:
            if not c in model.wv.vocab:
                print("Warning: word [%s] does not exist in Word2Vec vocabulary." % c)

    return inferred_vector

def get_document_vectors(questions, model, warning=False):
    document_vectors = []
    for question in questions:
        inferred_vector = get_document_vector(question, model, warning)
        document_vectors.append(list(inferred_vector))

    return np.array(document_vectors)

## (4) コサイン類似検索の実行

質問文は、my-ope プロダクションの nosetests テストケースから引用しました。

In [7]:
from sklearn.metrics.pairwise import cosine_similarity
from learning.core.datasource import Datasource
import time

def search_simiarity(question, dbow_model):
    '''
        質問文間でコサイン類似度を算出して、近い質問文の候補を取得する
        
        仕様はプロダクションに準拠しています
        ただし、文書のベクトル化は、TF-IDFではなく、
        Doc2Vecを使用します。
    '''
    start = time.time()

    datasource = Datasource('csv')
    question_answers = datasource.question_answers_for_suggest(_bot_id, question)

    #all_array = TextArray(question_answers['question'], vectorizer=self.vectorizer)
    #question_array = TextArray([question], vectorizer=self.vectorizer)
    all_array      = get_document_vectors(question_answers['question'], dbow_model)
    question_array = get_document_vectors([question], dbow_model, warning=True)
    
    print('count: my-ope all questions=%d, document vectors=%d (features=%d)' % (
        np.size(question_answers['question']), all_array.shape[0], all_array.shape[1]
    ))    
    print('count: question=%d, document vectors=%d (features=%d)' % (
        np.size([question]), question_array.shape[0], question_array.shape[1]
    ))    

    similarities = cosine_similarity(all_array, question_array)
    similarities = similarities.flatten()

    ordered_result = list(map(lambda x: {
        'question_answer_id': float(x[0]), 'similarity': x[1], 'answer_id': x[2]
    }, sorted(zip(question_answers['id'], similarities, question_answers['answer_id']), key=lambda x: x[1], reverse=True)))

    df = pd.DataFrame.from_dict(ordered_result)

    print(df[0:20])
    elapsed_time =  time.time() - start
    print("elapsed %d seconds" % elapsed_time)


In [8]:
# 正解＝6803
search_simiarity('JAL マイレージ', loaded_model_dbow)

2017/05/31 AM 11:38:57 ['./fixtures/learning_training_messages/benefitone.csv', './fixtures/learning_training_messages/ptna.csv', './fixtures/learning_training_messages/septeni.csv', './fixtures/learning_training_messages/toyotsu_human.csv']
2017/05/31 AM 11:38:57 ['./fixtures/question_answers/toyotsu_human.csv']


count: my-ope all questions=317, document vectors=317 (features=200)
count: question=1, document vectors=1 (features=200)
    answer_id  question_answer_id  similarity
0        6803             13378.0    0.616117
1        6775             13348.0    0.584991
2        6777             13350.0    0.579579
3        6890             13464.0    0.541347
4        6868             13442.0    0.538769
5        6774             13346.0    0.538247
6        6774             13347.0    0.522670
7        6735             13306.0    0.509320
8        6804             13379.0    0.507522
9        6809             13337.0    0.505009
10       6850             13424.0    0.504672
11       6999             13579.0    0.500682
12       6801             13376.0    0.500465
13       6790             13364.0    0.495466
14       6833             13407.0    0.493634
15       7021             13600.0    0.490861
16       6811             13385.0    0.488157
17       6738             13309.0    0.487759
18  

In [9]:
# 正解＝6763
search_simiarity('海外の出張費の精算の方法は？', loaded_model_dbow)

2017/05/31 AM 11:39:05 ['./fixtures/learning_training_messages/benefitone.csv', './fixtures/learning_training_messages/ptna.csv', './fixtures/learning_training_messages/septeni.csv', './fixtures/learning_training_messages/toyotsu_human.csv']
2017/05/31 AM 11:39:05 ['./fixtures/question_answers/toyotsu_human.csv']


count: my-ope all questions=317, document vectors=317 (features=200)
count: question=1, document vectors=1 (features=200)
    answer_id  question_answer_id  similarity
0        6763             13335.0    0.793270
1        6876             13450.0    0.728564
2        6830             13404.0    0.714666
3        6745             13317.0    0.691611
4        6856             13430.0    0.685585
5        6762             13334.0    0.684438
6        6868             13442.0    0.679503
7        6743             13314.0    0.672276
8        6827             13401.0    0.671643
9        6824             13398.0    0.669952
10       6863             13437.0    0.666715
11       6740             13311.0    0.665067
12       6749             13310.0    0.654500
13       6744             13316.0    0.651408
14       6902             13476.0    0.650846
15       6733             13304.0    0.647228
16       6898             13472.0    0.638212
17       6802             13377.0    0.634993
18  

In [10]:
# 正解＝6767
search_simiarity('VISAの勘定科目がわからない', loaded_model_dbow) 

2017/05/31 AM 11:39:13 ['./fixtures/learning_training_messages/benefitone.csv', './fixtures/learning_training_messages/ptna.csv', './fixtures/learning_training_messages/septeni.csv', './fixtures/learning_training_messages/toyotsu_human.csv']
2017/05/31 AM 11:39:13 ['./fixtures/question_answers/toyotsu_human.csv']


count: my-ope all questions=317, document vectors=317 (features=200)
count: question=1, document vectors=1 (features=200)
    answer_id  question_answer_id  similarity
0        6787             13360.0    0.721095
1        6767             13339.0    0.696659
2        6797             13372.0    0.686938
3        6900             13474.0    0.683477
4        6799             13374.0    0.678541
5        6796             13371.0    0.674079
6        6772             13361.0    0.648443
7        6893             13467.0    0.620368
8        6782             13355.0    0.610084
9        6791             13366.0    0.608042
10       6790             13365.0    0.602482
11       6794             13369.0    0.596453
12       6889             13463.0    0.576385
13       6890             13464.0    0.569637
14       6790             13364.0    0.566126
15       6781             13354.0    0.565036
16       6795             13370.0    0.559821
17       6801             13376.0    0.553004
18  

In [11]:
# 正解＝6909
search_simiarity('子供が生まれた', loaded_model_dbow) 

2017/05/31 AM 11:39:22 ['./fixtures/learning_training_messages/benefitone.csv', './fixtures/learning_training_messages/ptna.csv', './fixtures/learning_training_messages/septeni.csv', './fixtures/learning_training_messages/toyotsu_human.csv']
2017/05/31 AM 11:39:22 ['./fixtures/question_answers/toyotsu_human.csv']


count: my-ope all questions=317, document vectors=317 (features=200)
count: question=1, document vectors=1 (features=200)
    answer_id  question_answer_id  similarity
0        6909             13483.0    0.637177
1        6870             13444.0    0.541225
2        6777             13350.0    0.512139
3        6924             13503.0    0.486039
4        6871             13445.0    0.485358
5        7021             13600.0    0.472638
6        7009             13589.0    0.468125
7        6774             13346.0    0.466784
8        6776             13349.0    0.460481
9        6836             13410.0    0.458075
10       6913             13491.0    0.457961
11       6775             13348.0    0.457753
12       6846             13420.0    0.457714
13       6774             13347.0    0.454071
14       6990             13570.0    0.451012
15       6980             13560.0    0.450422
16       7007             13587.0    0.441555
17       6877             13451.0    0.439786
18  